In [1]:
!pip uninstall -y torch transformers
!pip install torch  --index-url https://download.pytorch.org/whl/cpu
!pip install -U tritonclient[all] transformers ovmsclient

Found existing installation: torch 2.2.1+cpu
Uninstalling torch-2.2.1+cpu:
  Successfully uninstalled torch-2.2.1+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.2.1%2Bcpu-cp310-cp310-linux_x86_64.whl (186.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.17.1 requires transformers[sentencepiece]>=4.26.0, which is not installed.
optimum-intel 1.16.0.dev0 requires transformers<4.39.0,>=4.36.0, which is not installed.
sentence-transformers 2.5.1 requires transformers<5.0.0,>=4.32.0, which is not installed.
torchaudio 2.1.0+cpu requires torch==2.1.0, but you have torch 2.2.1+cpu which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.2.1+cpu which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.2.1+cpu which is incompatibl

In [2]:
from huggingface_hub import notebook_login, whoami

try:
    whoami()
    print('Authorization token already provided')
except OSError:
    notebook_login()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:98: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Error: User cancelled dialog'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Authorization token already provided


In [3]:
import tritonclient.grpc as grpcclient

client = grpcclient.InferenceServerClient("10.208.15.235:8080")

server_live = client.is_server_live()
server_ready = client.is_server_ready()
model_ready = client.is_model_ready("llm-llama")

print("server_live:", server_live)
print("server_ready:", server_ready)
print("model_ready:", model_ready)

server_live: True
server_ready: True
model_ready: True


In [4]:
model_metadata = client.get_model_metadata("llm-llama")
print(model_metadata)

name: "llm-llama"
versions: "1"
platform: "OpenVINO"
inputs {
  name: "attention_mask"
  datatype: "INT64"
  shape: -1
  shape: -1
}
inputs {
  name: "beam_idx"
  datatype: "INT32"
  shape: -1
}
inputs {
  name: "input_ids"
  datatype: "INT64"
  shape: -1
  shape: -1
}
inputs {
  name: "position_ids"
  datatype: "INT64"
  shape: -1
  shape: -1
}
outputs {
  name: "logits"
  datatype: "FP32"
  shape: -1
  shape: -1
  shape: 256000
}



In [ ]:
import numpy as np
import os
import torch

from ovmsclient import make_grpc_client
from transformers import AutoTokenizer

SEED = 0
URL = "10.208.15.235:8080"
ACTOR = "general-knowledge"
QUESTION = "describe barack obama"

np.random.seed(SEED)
client = make_grpc_client(URL)

GENERAL_PRE_PROMPT = """You are super-human who is able to answer any question. All questions and answers should end with [EOS].
Question: What is the capital of Poland? [EOS]
Answer: Warsaw. [EOS]

Question: <Q> [EOS]
Answer:"""

PROGRAMMER_PRE_PROMPT = """You are super-programmer who is brilliant python coder. All the answers contain code snippets ending with [EOS].
Question: Write short python function to sum numbers in a list. [EOS]
Answer:
def sum_all_numbers(numbers):
  result = 0
    for num in numbers:
        result += num
    return result [EOS]

Question: <Q> [EOS]
Answer:"""

if ACTOR == 'general-knowledge':
    PRE_PROMPT = GENERAL_PRE_PROMPT
else:
    PRE_PROMPT = PROGRAMMER_PRE_PROMPT

PRE_PROMPT = PRE_PROMPT.replace("<Q>", QUESTION)

def convert_tensors_to_numpy(inputs):
    result = {}
    for key, tensor in inputs.items():
        result[key] = tensor.cpu().numpy()  # Move to CPU and convert to NumPy
    return result

def prepare_preprompt_kv_cache(preprompt):
    res = tokenizer(preprompt, return_tensors="pt", add_special_tokens=False)

    sequence_length = res.input_ids.size(1)
    position_ids = torch.arange(sequence_length).unsqueeze(0).long()

    beam_width = 4  # Adjust based on your beam search parameters
    beam_idx = torch.arange(sequence_length).unsqueeze(0).repeat(beam_width, 1)

    inputs = dict(
        input_ids = res.input_ids,                  # [1,X]
        attention_mask = res.attention_mask,        # [1,X]
        beam_idx = beam_idx,
        position_ids = position_ids
    )
    # for i in range(32):
    #     inputs[f"past_key_values.{i}.key"] = np.zeros((1, 32, 0, 128), dtype=np.float32)
    #     inputs[f"past_key_values.{i}.value"] = np.zeros((1, 32, 0, 128), dtype=np.float32)

    inputs = convert_tensors_to_numpy(inputs)

    inputs["beam_idx"] = inputs["beam_idx"].astype(np.int32)

    return client.predict(inputs=inputs, model_name='llm-llama')


def generate_next_inputs(previous_result, number_of_previous_tokens):
    probs = previous_result['logits'][0, -1, :]             # 1,N,32000

    probs = np.exp(probs)/sum(np.exp(probs))                # softmax
    next_token = np.random.choice(len(probs), p=probs)


    next_inputs = dict(
        input_ids = np.array([next_token], dtype=np.int64).reshape((1,1)),
        attention_mask =np.ones((1, number_of_previous_tokens+1), dtype=np.int64)
    )
    for j in range(32):
        next_inputs[f"past_key_values.{j}.key"] = previous_result[f"present.{j}.key"]
        next_inputs[f"past_key_values.{j}.value"] = previous_result[f"present.{j}.value"]
    return next_inputs, next_token

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf', trust_remote_code=True)
actual_token_list = list(tokenizer(PRE_PROMPT, return_tensors="pt", add_special_tokens=False).input_ids[0])
content_so_far = PRE_PROMPT

results = prepare_preprompt_kv_cache(PRE_PROMPT)
while True:
    next_inputs, token = generate_next_inputs(results, len(actual_token_list))
    results = client.predict(inputs=next_inputs, model_name='llama')

    actual_token_list.append(token)
    actual_content = tokenizer.decode(actual_token_list)

    print(actual_content.replace(content_so_far, ''), end='', flush=True)
    content_so_far = actual_content

    if actual_content.endswith("[EOS]"):
        break